A. Here are 400 random bits: 0000001101100100101000101100100110100000101001100100111101010000011111110100110101101000010111000100110111101101000110001101011011011000011010101100101111100011110010011110011011111010111000110110000100010111010110001111100011100111011101101111001110111110100101100010100100011110100010011110000111110011010101010000100001110101000101110110001001011011011111100111111110110101000000111111111110010100

1) Turn them into 200 random moves in the SmallGridWorld pictured below, in which there are two terminal states in gray and every move has reward -1: up=00, down=11, left=10, right=01.
2) Turn the 200 moves into episodes, each starting in state 5, the last episode possibly not terminating.
3) Starting with all state values equal to -18, apply first-visit Monte Carlo and Temporal Difference TD(0) with alpha = 0.1.
4) Visualize the updated state values after each episode.
5) Plot the root-mean squared error vs. episode number: $$RMS = \sqrt{\frac{1}{14} \sum_s \left( V(s) - v_\pi(s) \right)^2 }$$ where pi is the random policy.

![td.png](td.png)

In [1]:
action_string = "0000001101100100101000101100100110100000101001100100111101010000011111110100110101101000010111000100110111101101000110001101011011011000011010101100101111100011110010011110011011111010111000110110000100010111010110001111100011100111011101101111001110111110100101100010100100011110100010011110000111110011010101010000100001110101000101110110001001011011011111100111111110110101000000111111111110010100"
actions = []
for a in action_string:
    actions.append(int(a))

In [2]:
import numpy as np
states = np.zeros(14)
alpha = 0.1
reward = -1
# up=00, down=11, left=10, right=01
actions = ["00", "11", "10", "01"]

for s in states:
    s = -18

## First-Visit Monte Carlo

In [9]:
returns = np.zeros((14, 4), dtype=object)
q_values = np.zeros((14, 4))
epsilon = 0.1


def e_soft_policy(q_values, s, epsilon):
  if np.random.random() < epsilon:
    return np.random.choice(actions)
  else:
    return np.argmax(q_values) # choose the greedy action

def generate_episode():
    curr_action = e_soft_policy(q_values, states, epsilon)
    return curr_action

def first_visit_monte_carlo(episode):
    return

## Temporal Difference TD(0)

In [ ]:
def td_zero_prediction(env, policy, num_episodes, alpha, gamma):
    return


def policy(state):
    return 